# Using Jinja2 Functions in EDSL

This notebook demonstrates how to use the new `QuestionJinjaFunction` class in EDSL, which allows you to define computational logic using Jinja2 macros instead of Python functions with RestrictedPython.

## Advantages of Jinja2 Functions

- **Safe Serialization**: Unlike Python functions, Jinja2 macros can be safely serialized to JSON and deserialized.
- **Enhanced Security**: Jinja2 provides a restricted execution environment with limited capabilities.
- **Template Integration**: Natural integration with other template-based systems in EDSL.
- **Portability**: Macros can be transferred between systems without code execution concerns.

## Basic Usage

In [ ]:
# Import required modules
from edsl import Scenario, Agent, Results
from edsl.questions import QuestionJinjaFunction

### Creating a Simple Jinja2 Function

Let's start with a simple example: a function that adds two numbers from a scenario.

In [ ]:
# Define a Jinja2 macro template
template_str = """
{% macro add_numbers(scenario, agent_traits) %}
    {{ scenario.get("a", 0) + scenario.get("b", 0) }}
{% endmacro %}
"""

# Create the question
question = QuestionJinjaFunction(
    question_name="add_numbers",
    jinja2_template=template_str,
    macro_name="add_numbers",
    question_text="Add two numbers from the scenario",
    question_presentation="This is a computational question that adds two numbers."
)

# Create a scenario with the numbers to add
scenario = Scenario({"a": 5, "b": 10})

# Run the function
results = question.by(scenario).run(disable_remote_cache=True, disable_remote_inference=True)

# Display the result
print(f"Result: {results.select('answer.*').to_list()[0]}")

## More Complex Example with Conditional Logic

Now let's create a more complex function that can perform different operations based on agent traits.

In [ ]:
# Define a more complex Jinja2 macro template with conditional logic
template_str = """
{% macro process_numbers(scenario, agent_traits) %}
    {% set numbers = scenario.get("numbers", []) %}
    {% set operation = agent_traits.get("operation", "sum") %}
    {% set result = 0 %}
    
    {% if operation == "sum" %}
        {% for num in numbers %}
            {% set result = result + num %}
        {% endfor %}
        The sum is {{ result }}
    {% elif operation == "product" %}
        {% set result = 1 %}
        {% for num in numbers %}
            {% set result = result * num %}
        {% endfor %}
        The product is {{ result }}
    {% elif operation == "average" %}
        {% if numbers|length > 0 %}
            {% for num in numbers %}
                {% set result = result + num %}
            {% endfor %}
            {% set average = result / numbers|length %}
            The average is {{ average }}
        {% else %}
            No numbers to average
        {% endif %}
    {% else %}
        Unknown operation: {{ operation }}
    {% endif %}
{% endmacro %}
"""

# Create the question
complex_question = QuestionJinjaFunction(
    question_name="process_numbers",
    jinja2_template=template_str,
    macro_name="process_numbers",
    question_text="Process a list of numbers according to the specified operation",
    question_presentation="This question demonstrates conditional logic in Jinja2 functions."
)

# Create a scenario with a list of numbers
scenario = Scenario({"numbers": [1, 2, 3, 4, 5]})

# Try different operations with different agent traits
for operation in ["sum", "product", "average", "unknown"]:
    agent = Agent(traits={"operation": operation})
    results = complex_question.by(scenario).by(agent).run(disable_remote_cache=True, disable_remote_inference=True)
    print(f"Operation '{operation}': {results.select('answer.*').to_list()[0]}")

## Converting Python Functions to Jinja2 Macros

You can also convert simple Python functions to Jinja2 macros using the `func_for_conversion` parameter.

In [ ]:
# Define a Python function to convert
def calculate_discount(scenario, agent_traits):
    base_price = scenario.get("price", 0)
    quantity = scenario.get("quantity", 1)
    discount_percent = agent_traits.get("discount", 0) if agent_traits else 0
    
    total = base_price * quantity
    discount = total * (discount_percent / 100)
    final_price = total - discount
    
    return final_price

# Create the question using Python function conversion
converted_question = QuestionJinjaFunction(
    question_name="calculate_discount",
    func_for_conversion=calculate_discount,
    question_text="Calculate the final price after discount",
    question_presentation="This question demonstrates Python to Jinja2 conversion."
)

# Show the generated Jinja2 template
print("Generated Jinja2 template:")
print(converted_question.jinja2_template)

# Test the converted function
scenario = Scenario({"price": 100, "quantity": 5})
agent = Agent(traits={"discount": 20})

results = converted_question.by(scenario).by(agent).run(disable_remote_cache=True, disable_remote_inference=True)
print(f"\nDiscounted price: {results.select('answer.*').to_list()[0]}")

## Using Jinja2 Functions in Surveys

You can include Jinja2 functions in surveys just like any other question type.

In [ ]:
from edsl import Survey
from edsl.questions import QuestionFreeText, QuestionMultipleChoice

# Create a survey with a mixture of question types
survey = Survey()

# Add some regular questions
q1 = QuestionMultipleChoice(
    question_name="purchase_type",
    question_text="What are you purchasing?",
    question_options=["Electronics", "Clothing", "Groceries"]
)

q2 = QuestionFreeText(
    question_name="purchase_amount",
    question_text="How much are you spending?"
)

# Add a Jinja2 function to calculate tax based on purchase type
tax_calculator_template = """
{% macro calculate_tax(scenario, agent_traits) %}
    {% set purchase_type = scenario.get("purchase_type.answer", "") %}
    {% set amount = scenario.get("purchase_amount.answer", "0") | float %}
    
    {% if purchase_type == "Electronics" %}
        {% set tax_rate = 0.08 %}
    {% elif purchase_type == "Clothing" %}
        {% set tax_rate = 0.05 %}
    {% elif purchase_type == "Groceries" %}
        {% set tax_rate = 0.02 %}
    {% else %}
        {% set tax_rate = 0.06 %}
    {% endif %}
    
    {% set tax_amount = amount * tax_rate %}
    {% set total = amount + tax_amount %}
    
    The tax rate for {{ purchase_type }} is {{ (tax_rate * 100) | round(1) }}%.
    Tax amount: ${{ tax_amount | round(2) }}
    Total with tax: ${{ total | round(2) }}
{% endmacro %}
"""

q3 = QuestionJinjaFunction(
    question_name="tax_calculation",
    jinja2_template=tax_calculator_template,
    macro_name="calculate_tax",
    question_text="Tax Calculation",
    question_presentation="This question calculates the tax based on your purchase."
)

# Add the questions to the survey
survey.add_question(q1)
survey.add_question(q2)
survey.add_question(q3)

# The survey can now be used with any model or agent
print("Survey with Jinja2 function created successfully!")

## Serialization and Deserialization

One of the key benefits of Jinja2 functions is that they can be safely serialized and deserialized.

In [ ]:
from edsl.questions import QuestionBase
import json

# Create a simple Jinja2 function
template_str = """
{% macro greet(scenario, agent_traits) %}
    Hello, {{ agent_traits.get("name", "User") }}!
    {% if scenario.get("time_of_day") == "morning" %}
        Have a great day ahead!
    {% elif scenario.get("time_of_day") == "evening" %}
        Have a peaceful night!
    {% else %}
        Have a wonderful time!
    {% endif %}
{% endmacro %}
"""

greeting_question = QuestionJinjaFunction(
    question_name="greeting",
    jinja2_template=template_str,
    macro_name="greet",
    question_text="Greeting based on time of day",
    question_presentation="This question demonstrates serialization capabilities."
)

# Serialize to a dictionary
serialized = greeting_question.to_dict()
print("Serialized question:")
print(json.dumps(serialized, indent=2))

# Deserialize back to a question
deserialized_question = QuestionBase.from_dict(serialized)

# Verify that the deserialized question works correctly
scenario = Scenario({"time_of_day": "morning"})
agent = Agent(traits={"name": "John"})

results = deserialized_question.by(scenario).by(agent).run(disable_remote_cache=True, disable_remote_inference=True)
print(f"\nDeserialized question result: {results.select('answer.*').to_list()[0]}")

## Advanced Jinja2 Features

Jinja2 provides many advanced features that can be used in your function templates.

In [ ]:
# Advanced Jinja2 template with filters, macros, and more
advanced_template = """
{% macro analyze_data(scenario, agent_traits) %}
    {# Get the data from the scenario #}
    {% set data = scenario.get("data", []) %}
    {% set threshold = agent_traits.get("threshold", 50) %}
    
    {# Helper macro for formatting numbers #}
    {% macro format_number(num) %}
        {{ "%.2f"|format(num) }}
    {% endmacro %}
    
    {# Calculate statistics #}
    {% if data|length > 0 %}
        {% set total = 0 %}
        {% set above_threshold = 0 %}
        {% set max_val = data[0] %}
        {% set min_val = data[0] %}
        
        {% for val in data %}
            {% set total = total + val %}
            {% if val > threshold %}
                {% set above_threshold = above_threshold + 1 %}
            {% endif %}
            {% if val > max_val %}
                {% set max_val = val %}
            {% endif %}
            {% if val < min_val %}
                {% set min_val = val %}
            {% endif %}
        {% endfor %}
        
        {% set avg = total / data|length %}
        
        Data Analysis Report:
        - Count: {{ data|length }}
        - Average: {{ format_number(avg) }}
        - Maximum: {{ format_number(max_val) }}
        - Minimum: {{ format_number(min_val) }}
        - Values above threshold ({{ threshold }}): {{ above_threshold }}
        - Percentage above threshold: {{ format_number((above_threshold / data|length) * 100) }}%
    {% else %}
        No data to analyze.
    {% endif %}
{% endmacro %}
"""

analysis_question = QuestionJinjaFunction(
    question_name="data_analysis",
    jinja2_template=advanced_template,
    macro_name="analyze_data",
    question_text="Analyze a dataset",
    question_presentation="This question demonstrates advanced Jinja2 features."
)

# Test with some data
import random
data = [random.randint(10, 90) for _ in range(20)]
scenario = Scenario({"data": data})
agent = Agent(traits={"threshold": 50})

results = analysis_question.by(scenario).by(agent).run(disable_remote_cache=True, disable_remote_inference=True)
print(results.select('answer.*').to_list()[0])

## Best Practices for Jinja2 Functions

When using Jinja2 functions, keep these best practices in mind:

1. **Keep the logic simple**: Jinja2 is not designed for complex computations. Use it for simple transformations and formatting.

2. **Use comments**: Document your templates with Jinja2 comments `{# comment #}` to explain complex logic.

3. **Helper macros**: Create helper macros for reusable functionality within your main macro.

4. **Error handling**: Use conditional logic to handle missing data or edge cases gracefully.

5. **Type conversion**: Remember that all values in Jinja2 are strings unless explicitly converted using filters like `|int` or `|float`.

6. **Testing**: Test your Jinja2 functions with various inputs to ensure they handle all cases correctly.

## Limitations

While Jinja2 functions are powerful, they have some limitations compared to Python functions:

1. **Limited logic**: Complex algorithms are harder to implement in Jinja2.

2. **No external libraries**: You cannot import and use external libraries in Jinja2 macros.

3. **Performance**: For complex calculations, Jinja2 may be slower than native Python code.

4. **Debugging**: Error messages from Jinja2 can be less informative than Python exceptions.

5. **Whitespace handling**: Be careful with whitespace in your templates, as it can affect the output.

## Conclusion

The `QuestionJinjaFunction` class provides a safe and serializable alternative to `QuestionFunctional` with RestrictedPython. It's particularly useful for:

- Simple computational tasks that need to be serialized
- Template-based text generation with conditional logic
- Data formatting and basic analysis
- Integration with other template-based systems

By leveraging Jinja2's template engine, you can create functional questions that are both powerful and safely serializable.